In [2]:
import os
from time import sleep

import re
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

In [3]:
p_name = []
p_address = []
p_category = []
p_rating = []
p_review = []
p_tag = []
p_url = []

In [4]:
def main():
    global load_wb, review_num

    # 검색할 목록
    place_infos = ['울산 공원', '울산 볼링장', '울산 명소', '울산 만화카페', '울산 멀티방', '울산 방탈출', '울산 보드카페', '울산 코인노래방', '울산 숙소']

    for i, place in enumerate(place_infos):
        options = webdriver.ChromeOptions()
        options.add_argument('lang=ko_KR')
        chromedriver_path = "chromedriver"
        driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
        driver.implicitly_wait(10)
        driver.get('https://map.kakao.com/') 
        search(driver, place)
        driver.quit()
    print("finish")


def search(driver, place):

    search_area = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element(By.XPATH, '//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.find_element(By.XPATH, '//*[@id="info.main.options"]/li[1]/a').send_keys(Keys.ENTER)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(driver, place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element(By.XPATH, '//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            if i == 5:
                xPath = '//*[@id="info.search.page.next"]'
            driver.find_element(By.XPATH, xPath).send_keys(Keys.ENTER)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(driver, place, place_lists)
        
        for i in range(1,7):
          if i == 6:
            for i in range(3, 5):
              # 페이지 넘기기
              xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
              driver.find_element(By.XPATH, xPath).send_keys(Keys.ENTER)

              html = driver.page_source
              soup = BeautifulSoup(html, 'html.parser')
              place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

              crawling(driver, place, place_lists)
          else:
            for i in range(3, 7):
              # 페이지 넘기기
              xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
              if i == 6:
                  xPath = '//*[@id="info.search.page.next"]'
              driver.find_element(By.XPATH, xPath).send_keys(Keys.ENTER)

              html = driver.page_source
              soup = BeautifulSoup(html, 'html.parser')
              place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

              crawling(driver, place, place_lists)

    except:
        return
    finally:
        search_area.clear()


def crawling(driver, place_key, place_lists):
    while_flag = False
    for i, place in enumerate(place_lists):
        if place.select('.head_item > .tit_name > .link_name')[0]['title'] == '공원아파트':
            continue
            
        else:
            place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
            p_name.append(place_name)
            place_address = place.select('.info_item > .addr > p')[0].text  # place address
            p_address.append(place_address)
            place_rating = place.select('.rating > .score > em')[0].text
            p_rating.append(place_rating)
            try:
                place_category = place.select('span.subcategory.clickable')[0].text
            except:
                place_category=''
            p_category.append(place_category)
            place_more = place.select('.moreview')[0]['href']
            p_url.append(place_more)
        

            detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
            driver.find_element(By.XPATH, detail_page_xpath).send_keys(Keys.ENTER)
            driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환

            reviews = extract_review(driver, place_key, place_name)
            p_review.append(reviews)
            tags = extract_tag(driver, place_name)
            p_tag.append(tags)

            driver.close()
            driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(driver, place_key, place_id):
    review = []
    try:
        if driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_g').text == '(0)':
            return '리뷰 없음'
        else:
            return f'./kakao_reviews/{place_key}/{place_id}.csv'
    except:
            print('리뷰 뽑는 중 오류')
            return '리뷰 없음'
    # review_df=pd.DataFrame(columns={'p_id','u_id','u_rate','review'})
    
    # while True:
    #     try:
    #         if driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > a').text == '후기 더보기':
    #             driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > a').click()
    #         else:
    #             break
    #     except:
    #         break
    #     sleep(0.5)
    
    # html = driver.page_source
    # soup = BeautifulSoup(html, 'html.parser')
    
    # try:
    #     review_lists = soup.find('ul', {'class':'list_evaluation'}).find_all('li')
    # except:
    #     return ''
    
    # p_name = []
    # u_name = []
    # u_rating = []
    # u_comment = []
    
    # for review_list in review_lists:
    #     try:
    #         user_id = review_list.find('a', {'class':'link_user'}).text
    #     except:
    #         user_id = ''
    #         continue
        
    #     try:
    #         value = review_list.find('div', {'class':'star_info'}).find('div').find('span').find('span')['style']
    #         value = value.replace('width:', '')
    #         value = value.replace('%;', '')
    #         user_rating = int(value)/20
    #     except:
    #         user_rating = ''
    #         continue
        
    #     try:
    #         user_comment = review_list.find('p', {'class':'txt_comment'}).find('span').text
    #     except:
    #         user_comment = ''
    #         continue
    #     p_name.append(place_id)
    #     u_name.append(user_id)
    #     u_rating.append(user_rating)
    #     u_comment.append(user_comment)
        
    #     print('place_id:', place_id)
    #     print('user_id:', user_id)
    #     print('score:', user_rating)
    #     print('comment:', user_comment)
    
    # review_df = pd.DataFrame({
    #     'place_id':p_name,
    #     'user_id':u_name,
    #     'score':u_rating,
    #     'comment':u_comment
    # },)

    # review_df.to_csv(f'./kakao_reviews/{place_key}/{place_id}.csv', index = False, header=True, encoding='utf-8-sig')
    # return f'./kakao_reviews/{place_key}/{place_id}.csv'

def extract_tag(driver, place_name):
    tags = []
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 첫 페이지 리뷰 목록 찾기
        tag_lists = soup.select('.tag_g > a')

        tags = []
        # 리뷰가 있는 경우
        if len(tag_lists) != 0:
            for i, tag in enumerate(tag_lists):
                if len(tag) != 0:
                    tags.append(tag.text)

            return tags
        else:
            return '태그 없음'
    except:
        print('태그 뽑는 중 오류')
        return '태그 없음'

In [5]:
main()

C:\Users\82107\AppData\Local\Temp\ipykernel_31620\667134887.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기


리뷰 뽑는 중 오류
리뷰 뽑는 중 오류
리뷰 뽑는 중 오류
finish


In [6]:
import pandas as pd

In [22]:
print(len(p_name), len(p_address), len(p_rating), len(p_tag), len(p_review), len(p_url))

1260 1260 1260 1260 1260 1260


In [21]:
index = 1259
p_review.insert(index, '리뷰 없음')
p_tag.insert(index, '태그 없음')


In [ ]:
index = 1323
print(p_name[index], p_review[index])

해변의터마루 ./kakao_reviews/울산 숙소/해변의터마루.csv


In [20]:
for i in range(1326):
  try:
    if p_review[i] == '리뷰 없음':
      pass
    else:
      value = re.search('kakao_reviews/.+/(.+)\.csv', p_review[i]).group(1)
      if p_name[i] == value:
        continue
      else:
        print(i)
        break
  except:
    print(i)
    break

1259


In [ ]:
df = pd.DataFrame({
  'p_name':p_name,
  'p_address':p_address,
  'p_rating':p_rating,
  'p_category':p_category,
  'p_tag':p_tag,
  'p_review':p_review,
  'p_url':p_url
},)
df.to_csv('kakaoMap.csv',encoding='utf-8-sig')